In [ ]:
Name: Shreyas R. Dumbre
Roll No : 19
BE IT (B1)

In [1]:
"""We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells."""

'We are about to study the idea of a computational process.\nComputational processes are abstract beings that inhabit computers.\nAs they evolve, processes manipulate other abstract things called data.\nThe evolution of a process is directed by a pattern of rules\ncalled a program. People create programs to direct processes. In effect,\nwe conjure the spirits of the computer with our spells.'

In [7]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [8]:
torch.manual_seed(1)

In [9]:
# Implementing CBOW model for the exercise given by a tutorial in pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html
context_size = 2 # {w_i-2 ... w_i ... w_i+2}
embedding_dim = 10

In [10]:
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

In [11]:
def make_context_vector(context, word_to_idx):
    idxs = [word_to_idx[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

vocab = set(raw_text)
vocab_size = len(vocab)

word_to_idx = {word: i for i, word in enumerate(vocab)}
idx_to_word = {i: word for i, word in enumerate(vocab)}

data = []

In [12]:

for i in range(2, len(raw_text) - 2):
    context = [raw_text[i-2], raw_text[i-1],
               raw_text[i+1], raw_text[i+2]]
    target = raw_text[i]
    data.append((context, target))

In [ ]:
# Ge

In [13]:
class CBOW(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.proj = nn.Linear(embedding_dim, 128)
        self.output = nn.Linear(128, vocab_size)
        
    def forward(self, inputs):
        embeds = sum(self.embeddings(inputs)).view(1, -1)
        out = F.relu(self.proj(embeds))
        out = self.output(out)
        nll_prob = F.log_softmax(out, dim=-1)
        return nll_prob

In [14]:
model = CBOW(vocab_size, embedding_dim)
optimizer = optim.SGD(model.parameters(), lr=0.001)

losses = []
loss_function = nn.NLLLoss()

In [15]:
for epoch in range(100):
    total_loss = 0
    for context, target in data:
        context_vector = make_context_vector(context, word_to_idx)
        
        # Remember PyTorch accumulates gradients; zero them out
        model.zero_grad()
        
        nll_prob = model(context_vector)
        loss = loss_function(nll_prob, Variable(torch.tensor([word_to_idx[target]])))
        
        # backpropagation
        loss.backward()
        # update the parameters
        optimizer.step() 
        
        total_loss += loss.item()
        
    losses.append(total_loss)

print(losses)

[235.3547396659851, 231.2504210472107, 227.28084659576416, 223.43712449073792, 219.7141547203064, 216.10481572151184, 212.6047966480255, 209.2084927558899, 205.91152381896973, 202.70872175693512, 199.59591102600098, 196.56743896007538, 193.62000679969788, 190.75147688388824, 187.9560902118683, 185.2296940088272, 182.5699644088745, 179.97185349464417, 177.43296229839325, 174.94851350784302, 172.5169186592102, 170.1332973241806, 167.79425406455994, 165.49802088737488, 163.24142158031464, 161.02096807956696, 158.83659052848816, 156.68791508674622, 154.57073199748993, 152.48405319452286, 150.42698520421982, 148.39951944351196, 146.40119564533234, 144.42999005317688, 142.4861866235733, 140.56787306070328, 138.67332816123962, 136.80279511213303, 134.9571778178215, 133.13518822193146, 131.33669871091843, 129.56229543685913, 127.80784606933594, 126.07720935344696, 124.3686358332634, 122.68193072080612, 121.01412802934647, 119.36949115991592, 117.74674564599991, 116.14425528049469, 114.56068485

In [16]:
# Let's see if our CBOW model works or not

print("*************************************************************************")

context = ['process.','Computational','are', 'abstract']
context_vector = make_context_vector(context, word_to_idx)
a = model(context_vector).data.numpy()
print('Raw text: {}\n'.format(' '.join(raw_text)))
print('Test Context: {}\n'.format(context))
max_idx = np.argmax(a)
print('Prediction: {}'.format(idx_to_word[max_idx]))

*************************************************************************
Raw text: We are about to study the idea of a computational process. Computational processes are abstract beings that inhabit computers. As they evolve, processes manipulate other abstract things called data. The evolution of a process is directed by a pattern of rules called a program. People create programs to direct processes. In effect, we conjure the spirits of the computer with our spells.

Test Context: ['process.', 'Computational', 'are', 'abstract']

Prediction: the


In [17]:

context = ['processes','manipulate','abstract', 'things']
context_vector = make_context_vector(context, word_to_idx)
a = model(context_vector).data.numpy()
print('Raw text: {}\n'.format(' '.join(raw_text)))
print('Test Context: {}\n'.format(context))
max_idx = np.argmax(a)
print('Prediction: {}'.format(idx_to_word[max_idx]))

Raw text: We are about to study the idea of a computational process. Computational processes are abstract beings that inhabit computers. As they evolve, processes manipulate other abstract things called data. The evolution of a process is directed by a pattern of rules called a program. People create programs to direct processes. In effect, we conjure the spirits of the computer with our spells.

Test Context: ['processes', 'manipulate', 'abstract', 'things']

Prediction: other
